## Test order candidates

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import openpyxl
from rdkit.Chem.Draw import IPythonConsole, MolsToGridImage

#Show mols in dataframes
from rdkit.Chem import PandasTools
from rdkit import Chem
from rdkit.Chem.Draw import MolsToGridImage
from IPython.core.display import HTML
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import PandasTools
from rdkit.Chem import AllChem
from rdkit.Chem import rdDepictor
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import DataStructs
import os
from scipy.spatial.distance import pdist, squareform


Failed to find the pandas get_adjustment() function to patch
Failed to patch pandas - PandasTools will have limited functionality


In [2]:
TOP = os.getcwd().replace('notebooks', '')
raw_dir = TOP + 'data/raw/'
interim_dir = TOP + 'data/interim/'
external_dir = TOP + 'data/external/'
processed_dir = TOP + 'data/processed/'
figures_dir = TOP + 'reports/figures/'


In [5]:
df = pd.read_excel(interim_dir+'final_revised_universe_wmappingdict_191223.xlsx', sheet_name = 'universe', index_col = [0])


In [15]:
df.columns.tolist()

['dtxsid',
 'DSSTox_QC-Level',
 'Substance_Name',
 'Substance_CASRN',
 'Substance_Type',
 'smiles',
 'Structure_InChI',
 'Structure_InChIKey',
 'Structure_Formula',
 'Structure_MolWt',
 'Structure_SMILES_2D-QSAR',
 'Canonical_QSARr',
 'TSCA_STATUS',
 'chain_length',
 'category',
 'MolWeight',
 'LogP_pred',
 'LogP_predRange',
 'AD_LogP',
 'AD_index_LogP',
 'Conf_index_LogP',
 'MP_pred',
 'MP_predRange',
 'AD_MP',
 'AD_index_MP',
 'Conf_index_MP',
 'BP_pred',
 'BP_predRange',
 'AD_BP',
 'AD_index_BP',
 'Conf_index_BP',
 'LogVP_pred',
 'VP_predRange',
 'AD_VP',
 'AD_index_VP',
 'Conf_index_VP',
 'LogWS_pred',
 'WS_predRange',
 'AD_WS',
 'AD_index_WS',
 'Conf_index_WS',
 'LogHL_pred',
 'HL_predRange',
 'AD_HL',
 'AD_index_HL',
 'Conf_index_HL',
 'ID',
 'tsca_match',
 'subcategory',
 'final_centroid',
 'final_dist',
 'group',
 'MaxMin_picks',
 'tsca_centroid_tag',
 'tsca_centroid_id',
 'tsca_centroid_id.1',
 'tsca_analogues',
 'tsca_dist',
 'MaxMin_tsca_picks',
 'tsca_active_centroid_tag',


In [17]:
df[df['dtxsid'].isin(['DTXSID4041284','DTXSID6029177','DTXSID60862823','DTXSID7027831'])][['dtxsid', 'group','smiles','final_centroid',
 'final_dist',

 'MaxMin_picks',
 'tsca_centroid_tag',
 'tsca_centroid_id',
 'tsca_analogues',
 'tsca_dist',
 'MaxMin_tsca_picks',
 'tsca_active_centroid_tag',
 'tsca_active_centroid_id',
 'tsca_active_analogues',
 'tsca_active_dist',
 'MaxMin_tsca_active_picks',]]

,dtxsid,group,smiles,final_centroid,final_dist,MaxMin_picks,tsca_centroid_tag,tsca_centroid_id,tsca_analogues,tsca_dist,MaxMin_tsca_picks,tsca_active_centroid_tag,tsca_active_centroid_id,tsca_active_analogues,tsca_active_dist,MaxMin_tsca_active_picks
17,DTXSID7027831,"('FASA based PFAA precursors', 'gte7', nan, nan)",CN(CCO)S(=O)(=O)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(...,DTXSID6071908,0.509804,0,NaN,NaN,DTXSID7027831,0.391304,0.0,NaN,NaN,DTXSID7027831,0.391304,0.0
3144,DTXSID6029177,"('Other aliphatics', 'lt7', 1.0, 1.0)",FC(F)(F)C1(F)OC1(F)F,DTXSID6029177,0.000000,1,centroid,DTXSID6029177,DTXSID6029177,0.000000,1.0,NaN,NaN,DTXSID6029177,0.692308,0.0
5586,DTXSID60862823,"('Other aliphatics', 'lt7', 3.0, 2.0)",FC(=O)C(F)(OC(F)(F)C(F)(F)C(F)(F)F)C(F)(F)F,DTXSID2044630,0.342857,0,NaN,NaN,DTXSID60862823,0.378378,0.0,centroid,DTXSID60862823,DTXSID60862823,0.000000,1.0
15052,DTXSID4041284,"('unclassified', 'lt7', 1.0, nan)",C[N+](C)(CCCNS(=O)(=O)CCC(F)(F)C(F)(F)C(F)(F)C...,DTXSID001035193,0.594937,0,NaN,NaN,DTXSID4041284,0.740741,1.0,centroid,DTXSID4041284,DTXSID4041284,0.000000,0.0


|dtxsid|terminal category|Centroid/MaxMin|TSCA active Centroid/MaxMin|Comment|
|:--:|:---:|:----:|:---:|:----:|
|DTXSID7027831|('FASA based PFAA precursors', 'gte7', nan, nan)|No|No|0.39 to TSCA active centroid|
|DTXSID6029177|('Other aliphatics', 'lt7', 1.0, 1.0)|Yes|Yes|Existing TSCA pick|
|DTXSID60862823|('Other aliphatics', 'lt7', 3.0, 2.0)|No|Yes|Existing TSCA active pick|
|DTXSID4041284|('unclassified', 'lt7', 1.0, nan)|No|Yes|Existing TSCA active pick|